In [4]:
import os
import pandas as pd, numpy as np



In [6]:
#import delta
#import mimesis


In [14]:
import pyspark
from pyspark.sql import SparkSession


In [12]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('Desafio IBGE - Count.co') \
                    .getOrCreate()

24/02/18 13:25:51 WARN Utils: Your hostname, MacBookJulio.local resolves to a loopback address: 127.0.0.1; using 192.168.0.14 instead (on interface en0)
24/02/18 13:25:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/18 13:25:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/18 13:25:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [62]:
df = spark.read.option("sep", ";").option("linesep", "\n").option("quoteAll","True").format("csv").load("file:////Users/dal/Downloads/tabela5938.csv")

In [71]:
df.collect()

[Row(_c0='Tabela 5938 - Produto interno bruto a preços correntes, impostos, líquidos de subsídios, sobre produtos a preços correntes e valor adicionado bruto a preços correntes total e por atividade econômica, e respectivas participações - Referência 2010'),
 Row(_c0='Variável - Produto Interno Bruto a preços correntes (Mil Reais)'),
 Row(_c0='"Nível","Cód.","Brasil, Unidade da Federação e Município","Ano"\r'),
 Row(_c0='"Nível","Cód.","Brasil, Unidade da Federação e Município","2018","2019","2020","2021"\r'),
 Row(_c0='"BR","1","Brasil","7004141000","7389131000","7609597000","9012142000"\r'),
 Row(_c0='"UF","11","Rondônia","44913978","47091336","51598741","58170096"\r'),
 Row(_c0='"UF","12","Acre","15331123","15630017","16476371","21374440"\r'),
 Row(_c0='"UF","13","Amazonas","100109235","108181091","116019139","131531038"\r'),
 Row(_c0='"UF","14","Roraima","13369988","14292227","16024276","18202579"\r'),
 Row(_c0='"UF","15","Pará","161349602","178376984","215935604","262904979"\r'),


In [72]:
from pyspark.sql.functions import *

In [73]:
from functools import reduce

In [81]:
li_nivel = ['"BR"','"UF"','"MU"']


In [82]:
li_nivel

['"BR"', '"UF"', '"MU"']

In [84]:
filter_data_starts_with = reduce(
    lambda x, y: x | y,
    [col("_c0").startswith(s) for s in li_nivel], 
    lit(False))


In [85]:
#df_data = df.where(df._c0.isin(li_nivel))
df_data = df.where(filter_data_starts_with)

In [87]:
#df_data.show()

In [92]:
# é necesário renomear o arquivo gerado dentro do diretório tabela5938_clean.csv
df_data.write.csv("file:////Users/dal/Documents/desafio_ibge/count_sources/tabela5938_clean.csv", mode="overwrite" )

# Pandas

In [125]:
df_data2 = df_data.toPandas()

In [105]:
#define header row to pandas df
mun_header_cols = ["Nivel","Cod","EnteFederativo","2018","2019","2020","2021"]
#df_data2.columns = mun_header_cols

In [126]:
df_data2.columns = ['data']

Faz split dos dados , alterando os headers do Pandas df e drop na coluna original

In [128]:
df_data2[mun_header_cols] = df_data2["data"].str.split(',',expand=True)

In [134]:
df_data2.head(3)

,data,Nivel,Cod,EnteFederativo,2018,2019,2020,2021
0,"""BR"",""1"",""Brasil"",""7004141000"",""7389131000"",""7...","""BR""","""1""","""Brasil""","""7004141000""","""7389131000""","""7609597000""","""9012142000""\r"
1,"""UF"",""11"",""Rondônia"",""44913978"",""47091336"",""51...","""UF""","""11""","""Rondônia""","""44913978""","""47091336""","""51598741""","""58170096""\r"
2,"""UF"",""12"",""Acre"",""15331123"",""15630017"",""164763...","""UF""","""12""","""Acre""","""15331123""","""15630017""","""16476371""","""21374440""\r"


In [137]:
df_data2 = df_data2.drop(['data'],axis=1)

In [138]:
df_data2.head(2)

,Nivel,Cod,EnteFederativo,2018,2019,2020,2021
0,"""BR""","""1""","""Brasil""","""7004141000""","""7389131000""","""7609597000""","""9012142000""\r"
1,"""UF""","""11""","""Rondônia""","""44913978""","""47091336""","""51598741""","""58170096""\r"


Escreve os dados em formato CSV para ser usado no Count

In [140]:
df_data2.to_csv("/Users/dal/Documents/desafio_ibge/count_sources/tabela5938_clean_file.csv", columns=mun_header_cols, sep=",", encoding='UTF-8')

24/02/18 21:11:21 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2047733 ms exceeds timeout 120000 ms
24/02/18 21:11:21 WARN SparkContext: Killing executors is not supported by current scheduler.
